In [5]:
# off the shelf BERT from Huggingface
from transformers import BertModel, BertTokenizer
# numpy
import numpy as np
# pandas
import pandas as pd

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
'''
Read in .tsv of tagged sample data as a Pandas data frame
Add appropriate header to the columns as well.
'''
def read_as_df(filename):
    df = pd.read_csv(filename, sep="\t", header = None)
    df.columns = ["Text_Loc", "Sample", "Rating", "Specificity","Adj", "Adv", "Noun", "Verb", "Adp", "Time"]
    return df

In [9]:
samples = read_as_df("data/samples_data.tsv")

In [10]:
samples.head()

,Text_Loc,Sample,Rating,Specificity,Adj,Adv,Noun,Verb,Adp,Time
0,Text_Loc,Sample,Rating,Specificity,Adj,Adv,Noun,Verb,Adp,Time
1,../Gutenberg/samples/heart_of_darkness_20960_2...,...lap. she wore a starched white affair on he...,4.5,4.565217391304348,20,7,29,21,21,9
2,../Gutenberg/samples/adam_bede_601425_602225.txt,"...him, if he had known it, that the general a...",2.5,4.758620689655173,16,10,36,24,20,1
3,../Gutenberg/samples/middlemarch_1718514_17193...,"...passionate exclamation, as if some torture ...",2.0,2.9454545454545453,9,8,21,35,20,6
4,../Gutenberg/samples/tom_jones_1740540_1741340...,"...so vicious a passion from your heart, and y...",1.5,4.823529411764706,10,10,29,29,16,7


In [105]:
def tokenize_sample(sample):
    # tokenize
    return tokenizer(sample, return_tensors="pt")

In [ ]:
'''
Helper to add extra information to a 768-dimension BERT embedding.
'''
def augment_bert_embedding(bert_output, nums_to_add):
    temp = np.array(nums_to_add)
    augmented_bert_output = np.resize(bert_output, len(bert_output) + len(nums_to_add))
    augmented_bert_output[-len(nums_to_add):] = temp
    return augmented_bert_output

In [ ]:
def get_embeddings(df):
    
    for i in range (1, len(df) - 1):
        # get sample, remove leading + trailing ellipses
        sample = df['Sample'][i][3:-3]
        
        token_inputs = tokenize_sample(sample)

        # converts input_ids to their tokenized form
        # ie "insinuating" -> is word-pieced into in/##sin/##uating (3 diff tokens!)
        tokens=tokenizer.convert_ids_to_tokens(token_inputs["input_ids"][0])
        outputs = model(**token_inputs)
        
        # need to make a matrix for each sample
        for index, token in enumerate(tokens):
            # find the rep
            bert_embedding = outputs.last_hidden_state[0][index].detach().numpy()
            # augment w/ additional info
            augmented_embedding = augment_bert_embedding(bert_embedding, [df['Specificity'][i], df['Adp'][i]])    

In [128]:
get_embeddings(samples)

(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']
(770,)
(770,)
['4.565217391304348', '21']

(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.9454545454545453', '20']
(770,)
(770,)
['2.

(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']
(770,)
(770,)
['4.823529411764706', '16']

(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']
(770,)
(770,)
['4.428571428571429', '19']

(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']
(770,)
(770,)
['4.203389830508475', '17']

...

(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']
(770,)
(770,)
['5.283333333333333', '17']

(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.6818181818181817', '23']
(770,)
(770,)
['3.

(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.7916666666666665', '14']
(770,)
(770,)
['3.

(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']
(770,)
(770,)
['3.901639344262295', '17']

KeyboardInterrupt: 